In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pandas import plotting

#plotly 
import plotly.offline as py
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
init_notebook_mode(connected=True)
import plotly.figure_factory as ff
import plotly.express as px
# Mansno
import missingno as msno 
plt.style.use('fivethirtyeight')


In [25]:
df=pd.read_csv('../input/delhi-crime-data/Delhi Accident Data.csv')
df.head()

In [26]:
df=df.drop(["Unnamed: 7","Unnamed: 8","Unnamed: 9"], axis=1)

In [27]:
df.head()

In [28]:
df.isnull().sum()

In [29]:
df=df[df['TYPE OF ACCIDENT']!='NON INJURY']

In [30]:
target_count = df['TYPE OF ACCIDENT'].value_counts()
print('Class 0:', target_count[0])
print('Class 1:', target_count[1])
print('Proportion:', round(target_count[0] / target_count[1], 2), ': 1')

target_count.plot(kind='bar', title='Count (target)');

In [31]:
df.head()

In [32]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df['TYPE OF ACCIDENT']=label_encoder.fit_transform(df['TYPE OF ACCIDENT'])
df.head()

In [33]:
year_feature=pd.get_dummies(df, columns=['YEAR', 'DISTRICT'])
year_feature.head()

In [34]:
year_feature.columns

In [35]:
col=['YEAR_2008', 'YEAR_2009', 'YEAR_2010', 'YEAR_2011',
       'YEAR_2012', 'YEAR_2013', 'YEAR_2014', 'YEAR_2015', 'YEAR_2016',
       'YEAR_2017', 'DISTRICT_CENTRAL DELHI', 'DISTRICT_EAST DELHI',
       'DISTRICT_NEW DELHI', 'DISTRICT_NORTH DELHI(ROHINI)',
       'DISTRICT_NORTH EAST DELHI', 'DISTRICT_NORTH WEST DELHI',
       'DISTRICT_OUTER', 'DISTRICT_SHAHDARA', 'DISTRICT_SOUTH DELHI',
       'DISTRICT_SOUTH EAST DELHI', 'DISTRICT_SOUTH WEST DELHI',
       'DISTRICT_UNK', 'DISTRICT_WEST DELHI']

In [36]:
X=year_feature[col]
y=year_feature['TYPE OF ACCIDENT']

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [38]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=10)


In [39]:
lr=LogisticRegression()
lr.fit(X, y)

In [40]:
import statsmodels.api as sm
lr_model=sm.Logit(y, X)
result=lr_model.fit()
print(result.summary())

In [41]:
print(lr.intercept_)

In [42]:
y_pred=lr.predict(X_train)

In [43]:
print("Accuracy Score:-", metrics.accuracy_score(y_train, y_pred))
print("F1 Score:-", metrics.f1_score(y_train, y_pred))
print("Average Precision Score:-", metrics.average_precision_score(y_train, y_pred))
print("Log Loss:-", metrics.log_loss(y_train, y_pred))
print("Precision Score:-", metrics.precision_score(y_train, y_pred))
print("Recall Score:-", metrics.recall_score(y_train, y_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_train, y_pred))

In [44]:
y_test_pred=lr.predict(X_test)

In [45]:
lr_acc=metrics.accuracy_score(y_test, y_test_pred)
print("Accuracy Score:-",lr_acc)
print("F1 Score:-", metrics.f1_score(y_test, y_test_pred))
print("Average Precision Score:-", metrics.average_precision_score(y_test, y_test_pred))
print("Log Loss:-", metrics.log_loss(y_test, y_test_pred))
print("Precision Score:-", metrics.precision_score(y_test, y_test_pred))
print("Recall Score:-", metrics.recall_score(y_test, y_test_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_test, y_test_pred))

In [46]:
cfm=confusion_matrix(y_test, y_test_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cfm,
                               display_labels=lr.classes_)
disp.plot()

In [47]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix,classification_report,precision_score, plot_roc_curve, plot_precision_recall_curve, balanced_accuracy_score

def clf_scores(clf, y_predicted):
    # Accuracy
    acc_train = clf.score(X_train, y_train)*100
    acc_test = clf.score(X_test, y_test)*100
    
    roc = roc_auc_score(y_test, y_predicted)*100 
    tn, fp, fn, tp = confusion_matrix(y_test, y_predicted).ravel()
    cm = confusion_matrix(y_test, y_predicted)
    correct = tp + tn
    incorrect = fp + fn
    d=[acc_train, acc_test,  roc, correct, incorrect,  cm]
    index=["acc_train",'Test Accuracy',"Roc Score","COrrect","Incorrect","Confusion"  ]
    output=pd.DataFrame(data=d, index=index)
    
    d=sns.heatmap(cm, annot=True)
    dd=plot_roc_curve(clf, X_train, y_train)
    ddd=plot_precision_recall_curve(clf, X_train, y_train)

    return output,d, dd, ddd

In [48]:
#1. Logistic regression

from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression()
clf_lr.fit(X_train, y_train)

Y_pred_lr = clf_lr.predict(X_test)
print(clf_scores(clf_lr, Y_pred_lr))

In [49]:
from sklearn.linear_model import LogisticRegression
model= LogisticRegression()
model.fit(X_train, y_train)

In [50]:
threshold = []
accuracy = []

for p in np.unique(model.predict_proba(X_train)[:,1]):
    threshold.append(p)
    y_pred = (model.predict_proba(X_train)[:,1] >= p).astype(int)
    accuracy.append(balanced_accuracy_score(y_train,y_pred))

In [51]:
plt.figure(figsize=(10,6))
plt.scatter(threshold,accuracy)
plt.xlabel("Threshold")
plt.ylabel("Balanced accuracy")
plt.show()

# Build Logistic Regression with Hyperparameter

In [52]:
lr = LogisticRegression()


In [53]:
param_grid = [    
    {'penalty' : ['l1', 'l2', 'elasticnet'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]
    }
]

In [54]:
from sklearn.model_selection import GridSearchCV

In [55]:
clf = GridSearchCV(lr, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)

In [56]:
best_clf = clf.fit(X_train,y_train)

In [57]:
best_clf.best_estimator_

* The threshold probability should be 0.75.
* C=0.615848211066026

In [58]:
model=LogisticRegression(C=0.615848211066026)

In [59]:
model.fit(X_train, y_train)

In [60]:
y_pred =model.predict_proba(X_train)[:,1] >= 0.75

In [61]:
print("Accuracy Score:-", metrics.accuracy_score(y_train, y_pred))
print("F1 Score:-", metrics.f1_score(y_train, y_pred))
print("Average Precision Score:-", metrics.average_precision_score(y_train, y_pred))
print("Log Loss:-", metrics.log_loss(y_train, y_pred))
print("Precision Score:-", metrics.precision_score(y_train, y_pred))
print("Recall Score:-", metrics.recall_score(y_train, y_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_train, y_pred))

In [62]:
y_test_pred=model.predict(X_test)>= 0.75

In [63]:
lr_acc=metrics.accuracy_score(y_test, y_test_pred)
print("Accuracy Score:-",lr_acc)
print("F1 Score:-", metrics.f1_score(y_test, y_test_pred))
print("Average Precision Score:-", metrics.average_precision_score(y_test, y_test_pred))
print("Log Loss:-", metrics.log_loss(y_test, y_test_pred))
print("Precision Score:-", metrics.precision_score(y_test, y_test_pred))
print("Recall Score:-", metrics.recall_score(y_test, y_test_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_test, y_test_pred))

In [64]:
acc_train = model.score(X_train, y_train)*100
acc_test = model.score(X_test, y_test)*100
    
roc = roc_auc_score(y_test, y_test_pred)*100 
tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
cm = confusion_matrix(y_test, y_test_pred)
correct = tp + tn
incorrect = fp + fn
d=[acc_train, acc_test,  roc, correct, incorrect,  cm]
index=["acc_train",'Test Accuracy',"Roc Score","COrrect","Incorrect","Confusion"  ]
output=pd.DataFrame(data=d, index=index)
    
sns.heatmap(cm, annot=True)
plot_roc_curve(clf, X_train, y_train)
plot_precision_recall_curve(clf, X_train, y_train)